# vertical solar capture rate

This notebook is used to find the optimal solar panel orientation to maximize energy _volume_ vs energy _value_. Conventional wisdom is to point the panel north/south (towards the equator), sloped at an angle equal to the latitude you're at. This maximizes total MWh. However MWh of solar power are not very scarce on a sunny summer day at midday. It is scarce in evenings, and in winter. This means that the spot market price (i.e. the value to society, adjusted downwards by LGC subsidies) is much higher in winter, and in evenings. So why not point the panels more vertically, and more west? This notebook calculates the revenue from doing that in each NEM (Australian) region, for each possible angle.

To run this notebook, install the Python dependencies in `requirements.txt`. Then change `data_dir`, `nemosis_dir` and `results_dir` to suit your needs. (You can probably set `nemosis_dir = data_dir = results_dir`.) This notebook downloads the relevant price data from [`nemweb`](https://www.nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/).


The terms for orientation are:

* Azimuth:
  * horizontal compass orientation.
  * 0=north
  * 90=east

* Zenith
  * up-down
  * Sun:
    * 90 = sunset/sunrise
    * 0 = sun directly overhead (midday, if near the equator)
  * Panel:
    * 90=panel is vertical, facing horizon
    * 0=panel is horizontal, facing straight up

In [1]:
import datetime as dt
import os
import glob

import polars as pl
import astral
from astral.geocoder import database, lookup
import astral.sun
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz
from nemosis import dynamic_data_compiler, cache_compiler, static_table
from tqdm import tqdm

In [2]:
data_dir = '/home/matthew/Data/vertical-solar/'
nemosis_dir = '/home/matthew/Data/nemosis/' #os.path.join(data_dir, "nemosis")

results_dir = 'results'

In [3]:
start_time = dt.datetime(2015, 1, 1, 0, 5)
end_time = dt.datetime(2024, 12, 31, 23, 55)

file_dt_fmt = "%Y_%m_%d_%H_%M_%S"

nemosis_dt_fmt = "%Y/%m/%d %H:%M:%S"
aemo_dt_fmt = nemosis_dt_fmt

# number of degrees we check
# e.g. 0, 2, 4, ... 358 degrees
panel_azimuth_step = 2
panel_zenith_step = 2

results_pq_path = os.path.join(results_dir, f"results_full_{start_time.strftime(file_dt_fmt)}_to_{end_time.strftime(file_dt_fmt)}_{panel_azimuth_step=}_{panel_zenith_step}.parquet")

In [4]:
# map NEM regions to Astral cities
region_cities = {
   'VIC1': 'Melbourne',
   'TAS1': 'Hobart',
   'NSW1': 'Sydney', 
   'QLD1': 'Brisbane',
   'SA1': 'Adelaide'
}


In [5]:
intervals_per_h = 12

## Get Price Data

In [6]:
os.makedirs(data_dir, exist_ok=True)

In [ ]:
# cache file locally
# so when I'm on the train with bad wifi, it doesn't try to re-crawl nemweb

def escape_date(s):
    return s.split(' ')[0].replace('/', '_')

tables = ["DISPATCHPRICE", "ROOFTOP_PV_ACTUAL", "DISPATCH_UNIT_SCADA"]

for table in tables:

    print(f"Downloading {table} to {nemosis_dir}")
    cache_compiler(
        start_time.strftime(nemosis_dt_fmt),
        end_time.strftime(nemosis_dt_fmt),
        table, 
        nemosis_dir,
        fformat='parquet'
    )

INFO: Caching data for table DISPATCHPRICE
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRICE in date range already compiled in /home/matthew/Data/nemosis/.
INFO: Cache for DISPATCHPRI

In [ ]:
(
    pl.scan_parquet(os.path.join(nemosis_dir, "*_DISPATCHPRICE_*.parquet"))
    .with_columns(
        pl.col("SETTLEMENTDATE").str.strptime(pl.Datetime("ms"), aemo_dt_fmt)
    )
    .head().collect()
)
    

In [ ]:
# nem data is in 5 minute intervals
interval_min = 5
interval_sec = interval_min * 60

# polars timedelta expression for half an interval
half_offset = f"-{interval_sec//2}s"

price_lf = (
    pl.scan_parquet(os.path.join(nemosis_dir, "*_DISPATCHPRICE_*.parquet"))
    .with_columns(
        pl.col("SETTLEMENTDATE").str.strptime(pl.Datetime('ms'), aemo_dt_fmt)
    )
    .filter(pl.col("SETTLEMENTDATE").is_between(start_time, end_time))
    .filter(pl.col("INTERVENTION") == 0)
    .with_columns(
        pl.col("SETTLEMENTDATE").dt.replace_time_zone(time_zone="Australia/Brisbane")
    )
    .select(
        "REGIONID",
        pl.col("SETTLEMENTDATE").alias("INTERVAL_END"),
        pl.col("SETTLEMENTDATE").dt.offset_by(half_offset).alias("INTERVAL_MID"),
        pl.col("RRP").alias("PRICE")
    )
    #.with_columns(
    #    pl.col("REGIONID")
    #    .replace_strict(region_cities)
    #    .alias("CITY")
    #)
)
price_lf.head().collect()

## Get Sun Data

In [ ]:
observers = {}

for (region, city) in region_cities.items():
    observers[region] = lookup(city, database()).observer

In [ ]:
brisbane_tz = pytz.timezone('Australia/Brisbane')

def get_sun_angle(region, time):
    observer = observers[region]

    assert time.tzinfo is not None, "time zone not set up right"

    zenith = astral.sun.zenith(observer, time)
    azimuth = astral.sun.azimuth(observer, time)

    return (zenith, azimuth)



In [ ]:
data = []
price_df = price_lf.collect()
for row in tqdm(price_df.iter_rows(named=True), total = price_df.height, leave=False):
    region = row['REGIONID']
    time = row['INTERVAL_MID']
    (zenith, azimuth) = get_sun_angle(region, time)
    assert zenith >= 0, f"Negative zenith: {zenith=}"
    if zenith <= 90: # sun is up
        data.append({
            'REGIONID': region,
            'TIME': time,
            #'PRICE': row['PRICE'],
            'SUN': {
                'ZENITH': zenith,
                'AZIMUTH': azimuth
            }
        })

sun_lf = (
    pl.DataFrame(data)
    .lazy()
    .with_columns(
        pl.col("TIME").dt.cast_time_unit('ms')
    )
)
sun_lf.head().collect()

## Calculations

In [ ]:

# takes in a azimuth and zenith column
# returns a struct column, of an xyz vector
def az_zen_to_xyz(v):
    return pl.struct(
        # NORTH
        X = v.struct.field("ZENITH").radians().sin() *
            v.struct.field("AZIMUTH").radians().cos(),
        # EAST
        Y = v.struct.field("ZENITH").radians().sin() *
            v.struct.field("AZIMUTH").radians().sin(),
        # up
        Z = v.struct.field("ZENITH").radians().cos()
    )


# takes in two struct columns (x, y, z)
# returns cos(theta) of the angle between them
def dot_product(a, b):
    return (
        a.struct.field("X") * b.struct.field("X") +
        a.struct.field("Y") * b.struct.field("Y") +
        a.struct.field("Z") * b.struct.field("Z")
    )

# returns an angle between to cartesian vectors
def angle_between(a, b):
    return dot_product(a, b).arccos().degrees()

In [ ]:
# # range of orientations to test
# panel_azimuth_step = 10
# panel_azimuths = pl.int_range(0, 360, panel_azimuth_step, eager=False)

# panel_zenith_step = 10
# panel_zeniths = pl.int_range(0, 90+1, panel_zenith_step, eager=False)

# power_df = (
#     sun_lf
#     .lazy()
#     .join(
#         #pl.LazyFrame().select(PANEL_AZIMUTH=panel_azimuths),
#         pl.LazyFrame({"PANEL_AZIMUTH": panel_azimuths}),
#         how='cross'
#     )
#     .join(
#         pl.LazyFrame().select(PANEL_ZENITH=panel_zeniths),
#         how='cross'
#     )
#     .with_columns(
#         pl.struct(AZIMUTH="PANEL_AZIMUTH", ZENITH="PANEL_ZENITH").alias("PANEL")
#     )
#     .with_columns(
#         pl.col("PANEL").pipe(az_zen_to_xyz).alias("PANEL_XYZ"),
#         pl.col("SUN").pipe(az_zen_to_xyz).alias("SUN_XYZ"),
#     )
#     .with_columns(
#         pl.map_batches(
#             ["PANEL_XYZ", "SUN_XYZ"],
#             lambda row: dot_product(row[0], row[1]),
#             return_dtype=pl.Float64
#         )
#         .alias("COS_THETA")
#     )
#     .filter(
#         pl.col("COS_THETA") > 0
#     )
#     .rename({"COS_THETA": "POWER"})
#     .filter(
#         pl.col("SUN").struct.field("ZENITH") <= 90
#     )
# )

# power_df.head().collect()

In [ ]:
num_intervals = (
    price_lf
    .select(pl.col("INTERVAL_MID").n_unique())
    .collect()
    .item()
)
num_hours = num_intervals / intervals_per_h

In [ ]:
# Do each separate panel configuration in a loop
# collecting as we go
# otherwise we run out of memory (surprisingly)


results = []
for panel_azimuth in tqdm(range(0,360,panel_azimuth_step), leave=False):
    for panel_zenith in range(0,90+1,panel_zenith_step):
        df = (
            sun_lf
            .with_columns(
                pl.struct(AZIMUTH=pl.lit(panel_azimuth), 
                          ZENITH=pl.lit(panel_zenith)
                ).alias("PANEL")
            )
            .with_columns(
                pl.col("PANEL").pipe(az_zen_to_xyz).alias("PANEL_XYZ"),
                pl.col("SUN").pipe(az_zen_to_xyz).alias("SUN_XYZ"),
            )
            .with_columns(
                pl.map_batches(
                    ["PANEL_XYZ", "SUN_XYZ"],
                    lambda row: dot_product(row[0], row[1]),
                    return_dtype=pl.Float64
                )
                .alias("COS_THETA")
            )
            .filter(
                pl.col("COS_THETA") >= 0
            )
            .rename({"COS_THETA": "POWER"})
            .filter(
                pl.col("SUN").struct.field("ZENITH") <= 90
            )
            .rename({"TIME": "INTERVAL_MID"})
            .join(
                price_lf,
                on=["REGIONID", "INTERVAL_MID"],
                how='right'
            )
            .with_columns(
                pl.col("POWER").fill_null(strategy='zero')
            )
            .with_columns(
                (pl.col("POWER") / intervals_per_h).alias("ENERGY")
            )
            .with_columns(
                (pl.col("PRICE") * pl.col("ENERGY")).alias("REVENUE")
            )
            .group_by("REGIONID", pl.col("INTERVAL_MID").dt.year().alias("YEAR"))
            .agg(
                pl.col("REVENUE").sum(),
                pl.col("ENERGY").sum()
                    
            )
            .with_columns(
                # note that since we filtered some rows
                # we can't just do mean of POWER
                (pl.col("ENERGY") / num_hours).alias("POWER_AVG"),
                    
                (pl.col("REVENUE") / pl.col("ENERGY")).alias("CAPTURE_PRICE")
            )
            .with_columns(
                PANEL=pl.struct(
                          AZIMUTH=pl.lit(panel_azimuth), 
                          ZENITH=pl.lit(panel_zenith)),
                #PANEL_AZIMUTH=pl.lit(panel_azimuth),
                #PANEL_ZENITH=pl.lit(panel_zenith)
            )
            .collect()
        )
        results.append(df.lazy())

        null_rows = (
            df
            .filter(pl.col("PANEL").is_null())
        )
        assert null_rows.is_empty(), f"null panel orientation? {panel_azimuth=} {panel_zenith=}"

        

results_df = pl.concat(results, how='vertical').collect()
results_df.head()

In [ ]:
os.makedirs(results_dir, exist_ok=True)
results_df.write_parquet(results_pq_path)

## Graph

In [ ]:
# can skip straight to here and not re-run previous slow computations
results_df = pl.read_parquet(results_pq_path)

In [ ]:
plot_all_years_df = (
    results_df
    .with_columns(
        PANEL_AZIMUTH=pl.col("PANEL").struct.field("AZIMUTH"),
        PANEL_ZENITH=pl.col("PANEL").struct.field("ZENITH"),
    )
    # shift the starting angle, so the boring part of the plot as at the left/right wraparound
    .with_columns(
        PANEL_AZIMUTH = pl.col("PANEL_AZIMUTH") + pl.when(pl.col("PANEL_AZIMUTH") >= 180).then(pl.lit(-360)).otherwise(pl.lit(0))
    )
    .sort("REGIONID", "PANEL_AZIMUTH", "PANEL_ZENITH")
)

In [ ]:
# for the heatmap colours, only show the last year
plot_last_year_df = (
    plot_all_years_df 
    .filter(pl.col("YEAR") == pl.col("YEAR").max())
    .sort("REGIONID")
)

heat_colours_df = plot_last_year_df

In [ ]:
# find the setup that gives the maximum energy volume, capture price and revenue
cols = ["POWER_AVG", "CAPTURE_PRICE", "REVENUE"]
peaks = []
for col in cols:
    peaks.append(
        results_df
        .sort(col, descending=True)
        .group_by("REGIONID", "YEAR")
        .first()
        .select(
            "REGIONID",
            "YEAR",
            "PANEL",
            pl.col("PANEL").struct.field("AZIMUTH").alias("PANEL_AZIMUTH"),
            pl.col("PANEL").struct.field("ZENITH").alias("PANEL_ZENITH"),
            pl.lit(col).alias("MAX_OF"),
            *cols
        )
        # shift the starting angle, so the boring part of the plot is at the left/right wraparound
        .with_columns(
            PANEL_AZIMUTH = pl.col("PANEL_AZIMUTH") + pl.when(pl.col("PANEL_AZIMUTH") >= 180).then(pl.lit(-360)).otherwise(pl.lit(0))
        )
    )
    
peaks_all_years = (
    pl.concat(peaks, how='vertical')
    .sort("REGIONID", "YEAR")
    .with_columns(
        pl.when(pl.col("YEAR") == pl.col("YEAR").min())
        .then(pl.col("YEAR").cast(pl.String()))
        .when(pl.col("YEAR") == pl.col("YEAR").max())
        .then(pl.col("YEAR").cast(pl.String()))
        .otherwise(None)
        .alias("YEAR_FIRST_LAST_TEXT"),
        (pl.col("YEAR") - pl.col("YEAR").min() + 1).alias("YEAR_RELATIVE")
    )
)
peaks_last_year = (
    peaks_all_years
    .filter(pl.col("YEAR") == pl.col("YEAR").max())
)




In [ ]:
os.makedirs(results_dir, exist_ok=True)

plot_configs = [
    ("REVENUE", "Revenue", "$"),
    ("POWER_AVG", "Physical Output", "Average MW"),
    ("CAPTURE_PRICE", "Capture Price", "$/MWh"),
]

for (z, title, units) in plot_configs:
    
    fig = px.density_heatmap(heat_colours_df, 
                             x="PANEL_AZIMUTH", 
                             y="PANEL_ZENITH",
                             z=z,
                             title = f"{title} vs solar panel orientation (2024)",
                             histfunc="sum", # already one value per group
                             facet_col="REGIONID",
                             facet_col_wrap=3,
                             facet_row_spacing=0.1,
                             labels = {
                                 "PANEL_AZIMUTH": "azimuth (compass)",
                                 "PANEL_ZENITH": "zenith (tilt)",
                                 "REVENUE": "Revenue",
                                 "CAPTURE_PRICE": "Capture Price",
                                 "POWER_AVG": "Average Power",
                             },
                             hover_data=["YEAR", "REVENUE", "CAPTURE_PRICE", "POWER_AVG"],
                             color_continuous_scale="Turbo",
                            )
    
    fig.update_traces(xbins= {
                             'start': heat_colours_df.select(pl.col("PANEL_AZIMUTH").min()).item() - panel_azimuth_step/2,
                             'end': heat_colours_df.select(pl.col("PANEL_AZIMUTH").max()).item(),
                             'size': panel_azimuth_step
                         },
                         ybins = {
                             'start': heat_colours_df.select(pl.col("PANEL_ZENITH").min()).item() - panel_zenith_step / 2,
                             'end': heat_colours_df.select(pl.col("PANEL_ZENITH").max()).item(),
                             'size': panel_zenith_step
                         }
                        )

    # trace_fig = px.line(
    #     peaks_all_years.filter(pl.col("MAX_OF") == "REVENUE"),
    #     x="PANEL_AZIMUTH", 
    #     y="PANEL_ZENITH",
    #     text="YEAR_FIRST_LAST_TEXT",
    #     #size = "YEAR_RELATIVE",
    #     #color="YEAR",
    #     facet_col="REGIONID",
    #     hover_data=["YEAR", "REVENUE", "CAPTURE_PRICE", "POWER_AVG"],
    #     facet_col_wrap=3
    # )

        
    # # Add line traces to heatmap
    # for trace in trace_fig.data:
    #     fig.add_trace(trace)
    
    scatter_fig = px.scatter(peaks_last_year, 
                             x="PANEL_AZIMUTH", 
                             y="PANEL_ZENITH",
                             symbol="MAX_OF",
                             facet_col="REGIONID",
                             hover_data=["YEAR", "REVENUE", "CAPTURE_PRICE", "POWER_AVG"],
                             facet_col_wrap=3,
                             labels={
                                 "MAX_OF": "Maximum of what?"
                             }
                            )

    scatter_fig.update_traces(marker={'size': 13, 'color': 'black'})
    
    # Update legend labels
    scatter_fig.for_each_trace(
        lambda trace: trace.update(
            name={
                'CAPTURE_PRICE': 'weighted-average price',
                'REVENUE': 'revenue', 
                'POWER_AVG': 'energy volume'
            }.get(trace.name, trace.name)
        )
    )
    
    # Add scatter traces to heatmap
    for trace in scatter_fig.data:
        fig.add_trace(trace)
    
    fig.update_xaxes(
        tickmode='array',
        tickvals=[-180, -90, 0, 90, 180-1, 270],
        ticktext=['S', 'W', 'N', 'E', 'S', 'W']
    )
    
    fig.update_yaxes(
        tickmode='array',
        tickvals=[0, 90],
        ticktext=['h', 'v']
    )
    
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    # manually reposition legends, otherwise they overlap
    fig.update_layout(
        # dots
        legend=dict(
            orientation="v",
            yanchor="top",
            y=0.4,
            xanchor="center",
            x=0.85,
            title="Orientation which maximizes ..."
        ),
        # heatmap
        legend2=dict(
            orientation="v", 
            yanchor="top",
            y=1,
            xanchor="left", 
            x=1.02,
        )
    )

    fig.update_layout(width=900, height=500)
    
    fig.show()

    fig.write_image(os.path.join(results_dir, f"{z}.png"))

In [ ]:
improvement = (
    peaks_last_year
    .pivot("MAX_OF", values="REVENUE", index="REGIONID")
    .with_columns(
        (pl.col("REVENUE") / pl.col("POWER_AVG")).alias("IMPROVEMENT")
    )
    .sort("IMPROVEMENT")
)

improvement.write_parquet(os.path.join(results_dir, 'improvement.parquet'))
improvement.write_csv(os.path.join(results_dir, 'improvement.csv'))
improvement


## Compare to penetration

Plot angle between value and volume maximizing, vs % solar in the grid (including rooftop).

In [ ]:
# find the angle between two vectors
angle_diff_history = (
    peaks_all_years
    .pivot("MAX_OF", index=["YEAR", "REGIONID"], values="PANEL")
    .rename({
        "REVENUE": "PANEL_MAX_REVENUE",
        "POWER_AVG": "PANEL_MAX_POWER",
        "CAPTURE_PRICE": "PANEL_MAX_CAPTURE_PRICE",
    })
    .with_columns(
        pl.col("PANEL_MAX_REVENUE").pipe(az_zen_to_xyz).alias("REVENUE_XYZ"),
        pl.col("PANEL_MAX_POWER").pipe(az_zen_to_xyz).alias("POWER_XYZ"),
    )
    .with_columns(
        pl.map_batches(
            ["REVENUE_XYZ", "POWER_XYZ"],
            lambda row: dot_product(row[0], row[1]),
            return_dtype=pl.Float64
        )
        .alias("COS_THETA")
    )
    .with_columns(
        pl.col("COS_THETA").arccos().degrees().alias("THETA")
    )
    .sort("YEAR", "REGIONID")
)
angle_diff_history

In [ ]:
rooftop_pv_raw = (
    pl.scan_parquet(os.path.join(nemosis_dir, "*_ROOFTOP_PV_ACTUAL*.parquet"), low_memory=True)
    .with_columns(
        pl.col("INTERVAL_DATETIME").str.head(4).cast(pl.UInt16).alias("YEAR"),
        pl.col("INTERVAL_DATETIME", "LASTCHANGED").str.strptime(pl.Datetime("ms"), aemo_dt_fmt)
    )
    .filter(pl.col("INTERVAL_DATETIME").is_between(start_time, end_time))
)
rooftop_pv_raw.head().collect()

In [ ]:
#rooftop_pv_deduplicated_pq = os.path.join(data_dir, f'rooftop_dedup_{start_time.strftime(file_dt_fmt)}_to_{end_time.strftime(file_dt_fmt)}.parquet')

rooftop_pv = (
    rooftop_pv_raw
    .filter(pl.col("INTERVAL_DATETIME").is_between(start_time, end_time))
    .filter(pl.col("REGIONID").cast(str).str.ends_with("1")) # delete sub-regions
    # deduplicate
    .sort(by=["TYPE", "QI", "LASTCHANGED"], descending=[False, True, True])
    .group_by(["REGIONID", "YEAR", "INTERVAL_DATETIME"])
    .first()
    .select(["REGIONID", "YEAR", "INTERVAL_DATETIME", "POWER"])
    .rename({
        "INTERVAL_DATETIME": "INTERVAL_END",
        "POWER": "POWER_MW",
    })
    .with_columns(
        # convert power to energy, for half hour readings
        (pl.col("POWER_MW") / 2).alias("ENERGY_MWH")
    )
    .with_columns(pl.col("INTERVAL_END").dt.cast_time_unit("ms"))
    .with_columns([pl.col("INTERVAL_END").dt.offset_by("-30m").alias("INTERVAL_START")])
    .filter(pl.col("REGIONID").is_not_null()) # bug with polars adds null rows
)


In [ ]:
yearly_rooftop_df = (
    rooftop_pv
    .group_by("YEAR")
    .agg(pl.col("ENERGY_MWH").sum())
    .collect()
)

yearly_rooftop_df.head()

In [ ]:
dispatchscada = (
    pl.scan_parquet(os.path.join(nemosis_dir, "*_DISPATCH_UNIT_SCADA*.parquet"), low_memory=True)
    # .with_columns(
    #     pl.col("SETTLEMENTDATE").str.head(4).cast(pl.UInt16).alias("YEAR")
    # )
    # .with_columns(
    #     pl.col("SETTLEMENTDATE").str.strptime(pl.Datetime("ms"), aemo_dt_fmt)
    # )
    # .filter(pl.col("SETTLEMENTDATE").is_between(start_time, end_time))
    # .rename({
    #     "SETTLEMENTDATE": "INTERVAL_END"
    # })
    # .with_columns(
    #     pl.col("INTERVAL_END").dt.offset_by("-5m").alias("INTERVAL_START")
    # )
    .with_columns(
        pl.col("SETTLEMENTDATE").str.head(4).cast(pl.UInt16).alias("YEAR")
    )
)
dispatchscada.head().collect()

In [ ]:
static_pd = static_table('Generators and Scheduled Loads', nemosis_dir)
static_pd

In [ ]:
static = (
    pl.from_pandas(static_pd)
    .with_columns(
        (pl.col("Fuel Source - Primary") == "Solar").alias("IS_SOLAR"),
    )
    # remove loads and storage
    .filter(pl.col("Dispatch Type") == "Generating Unit")
    .filter(pl.col("Technology Type - Primary") != "Storage")
    .lazy()
)

In [ ]:
# sink to parquet file, then read back
# because otherwise we run out of memory
dispatchscada_processed_pq = os.path.join(data_dir, "dispatchscada_years.parquet")
(
    dispatchscada
    .with_columns(
        (pl.col("SCADAVALUE") / intervals_per_h).alias("ENERGY_MWH")
    )
    .join(static, on="DUID", how="inner")
    .select(
        pl.col("Region").alias("REGIONID"),
        "YEAR",
        "ENERGY_MWH",
        "IS_SOLAR",
    )
    .sink_parquet(dispatchscada_processed_pq)
)

In [ ]:
regionid_enum = pl.Enum(["QLD1", "NSW1", "VIC1", "SA1", "TAS1"])
regionid_enum

In [ ]:
solar_frac_history = (
    pl.concat([
        pl.scan_parquet(dispatchscada_processed_pq, low_memory=True),
        rooftop_pv
        .with_columns(pl.lit(True).alias("IS_SOLAR"))
    ], how='diagonal_relaxed')
    .cast({"REGIONID": regionid_enum})
    .group_by("YEAR", "REGIONID")
    .agg(
        (pl.col("ENERGY_MWH").filter(pl.col("IS_SOLAR")).sum() / pl.col("ENERGY_MWH").sum()).alias("FRAC_SOLAR")
    )
    .sort("REGIONID", "YEAR")
    .collect()
)
solar_frac_history

In [ ]:
angle_vs_pv_frac = (
    angle_diff_history.cast({"REGIONID": regionid_enum})
    .join(solar_frac_history, on=["YEAR", "REGIONID"], how='inner')
    .join(peaks_all_years.cast({"REGIONID": regionid_enum}), on=["YEAR", "REGIONID"], how='inner')
    .sort("REGIONID", "YEAR")
)
angle_vs_pv_frac

In [ ]:
fig = px.line(
    angle_vs_pv_frac,
    x="FRAC_SOLAR",
    y="THETA",
    color="REGIONID",
    hover_data=["YEAR", "REVENUE", "CAPTURE_PRICE", "POWER_AVG"],
    #facet_col_wrap=3
)

fig.update_layout(xaxis_tickformat = '.0%')

fig.show()
fig.write_image(os.path.join(results_dir, f"theta_vs_frac_solar.png"))

In [ ]:
(
    solar_frac_history
    .cast({"REGIONID": str})
    .filter(pl.col("YEAR") == pl.col("YEAR").max())
    .join(improvement, on=["REGIONID"])
    .sort("IMPROVEMENT")
)